In [330]:
import pandas as pd
import spotipy
import os
import requests
from datetime import datetime
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

load_dotenv()

CID = os.environ.get("CLIENT_ID")
SECRET = os.environ.get("CLIENT_SECRET")

In [331]:
def spotify_auth_obj():
    client_credentials_manager = SpotifyClientCredentials(client_id=CID, client_secret=SECRET)
    return spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [332]:
def get_albums(artist, spotify_obj):
    artist_id = spotify_obj.search(q=artist, type='artist')['artists']['items'][0]['id']
    return spotify_obj.artist_albums(artist_id=artist_id, limit=50, album_type='album')

In [444]:
def discography_df(albumns, spotify, end=None):
    track_list = []
    album_list = []
    pop_list = []
    
    date = end if end!=None else '2021'
    
    for album in albums['items'][::-1]:
        if album['release_date'][:4] < date:
            tracks = sp.album_tracks(album['id'])
            for track in tracks['items']:
                album_list.append(album['name'])
                track_list.append(track['name'])
                pop_list.append(sp.track(track['id'])['popularity'])
    
    return pd.DataFrame({       
        'album':album_list,
        'track':track_list,
        'popularity':pop_list})

In [445]:
sp = spotify_auth_obj()

In [473]:
albums = get_albums("Pink Floyd", sp)

In [474]:
discography = discography_df(albums, sp)

In [475]:
tmp_series = discography[['album', 'popularity']].groupby(by='album').sum()['popularity']

tmp_series = tmp_series.reindex(discography[['album', 'popularity']].groupby(by='album').sum().index)

In [476]:
discography['album_popularity'] = discography['album'].map(tmp_series)

In [477]:
discography['album_tracks'] = discography['album'].map(discography['album'].value_counts())

In [478]:
discography['avg_album_popularity'] = round(discography['album_popularity']/discography['album_tracks'], 2)

In [479]:
discography

,album,track,popularity,album_popularity,album_tracks,avg_album_popularity
0,"Amsterdamse Bos, Free Concert, Live, 26 June 1971","Careful With That Axe, Eugene - Amsterdamse Bo...",23,93,5,18.60
1,"Amsterdamse Bos, Free Concert, Live, 26 June 1971","Cymbaline - Amsterdamse Bos, Live 26 June 1971",21,93,5,18.60
2,"Amsterdamse Bos, Free Concert, Live, 26 June 1971",Set The Controls For The Heart Of The Sun - Am...,18,93,5,18.60
3,"Amsterdamse Bos, Free Concert, Live, 26 June 1971","A Saucerful Of Secrets - Amsterdamse Bos, Live...",16,93,5,18.60
4,"Amsterdamse Bos, Free Concert, Live, 26 June 1971","Embryo - Amsterdamse Bos, Live 26 June 1971",15,93,5,18.60
...,...,...,...,...,...,...
599,Delicate Sound of Thunder (2019 Remix) [Live],Money - 2019 remix [Live],29,709,23,30.83
600,Delicate Sound of Thunder (2019 Remix) [Live],"Another Brick In the Wall, Part 2 - 2019 remix...",29,709,23,30.83
601,Delicate Sound of Thunder (2019 Remix) [Live],Comfortably Numb - 2019 remix [Live],31,709,23,30.83
602,Delicate Sound of Thunder (2019 Remix) [Live],One Slip - 2019 remix [Live],29,709,23,30.83


In [480]:
discography[['album', 'avg_album_popularity']].drop_duplicates().sort_values('avg_album_popularity', ascending=False)

,album,avg_album_popularity
122,The Dark Side of the Moon,65.70
132,Wish You Were Here,65.40
137,Wish You Were Here (Remastered Version),60.60
112,The Dark Side Of The Moon (2011 Remastered Ver...,60.20
178,The Wall,57.81
147,Animals,55.20
86,Meddle,53.83
318,The Division Bell,53.82
152,The Wall (Remastered),52.42
329,The Division Bell (2011 Remastered Version),49.64


In [481]:
discography[['album', 'album_popularity']].drop_duplicates().sort_values('album_popularity', ascending=False)

,album,album_popularity
455,The Later Years,2321
389,The Endless River,1531
178,The Wall,1503
152,The Wall (Remastered),1363
364,Pulse (Live),1006
340,Pulse,901
230,"Pulse (Live, 2018 Remaster)",826
547,Delicate Sound Of Thunder (2019 Remix; Live),716
581,Delicate Sound of Thunder (2019 Remix) [Live],709
122,The Dark Side of the Moon,657
